In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import openai 


load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

--- 

# Data

In [2]:
DATA_FOLDER_PATH = os.path.join(os.getcwd(), "data")
assert os.path.exists(DATA_FOLDER_PATH)

In [4]:
# Load brand knowledge 
with open(os.path.join(DATA_FOLDER_PATH, "brand_data", "brand_knowledge.txt"), "r") as f:
    brand_knowledge = f.read()
    
# Load persona
with open(os.path.join(DATA_FOLDER_PATH, "persona", "persona.txt"), "r") as f:
    persona = f.read()
    
# Load copywriting guidelines
with open(os.path.join(DATA_FOLDER_PATH, "brand_data", "copywriting_guidelines.txt"), "r") as f:
    copywriting_guidelines = f.read()
    
# Load Plateform Specs
with open(os.path.join(DATA_FOLDER_PATH, "platform_specs", "hermes.txt"), "r") as f:
    hermes_specs = f.read()
    
with open(os.path.join(DATA_FOLDER_PATH, "platform_specs", "sephora.txt"), "r") as f:
    sephora_specs = f.read()

In [5]:
# Load product data
product_data = {}  

languages = os.listdir(os.path.join(DATA_FOLDER_PATH, "product_data"))

product_data = {
    language: {
        product[:-4].replace('_', ' ').capitalize().replace('hermes', 'Hermès'): open(os.path.join(DATA_FOLDER_PATH, "product_data", language, product), "r").read() for product in os.listdir(os.path.join(DATA_FOLDER_PATH, "product_data", language))
    } for language in languages
}
    
    

In [6]:
product_data

{'fr': {'Galop': "LA CRÉATION\nSigné par le parfumeur d'Hermès Christine Nagel en 2016, Galop d'Hermès s'inspire de l'univers équestre et incarne la liberté en mouvement : « Chez Hermès, j'ai découvert toute la féminité du cuir. J'ai composé Galop d'Hermès comme un tableau avec deux matières emblématiques d'Hermès et de la parfumerie : le cuir et la rose. »\n\nLES NOTES OLFACTIVES\nFloral et cuiré, le parfum Galop d'Hermès développe les notes charnues de la rose et celles souples du cuir, au côté d'un safran racé.\n\nL'OBJET\nDessiné par Philippe Mouquet, le flacon-étrier puise avec modernité dans la tradition équestre et l'histoire de la Maison.\n\nL'ÉTHIQUE\nLe flacon se remplit grâce à une recharge de 125 ml munie d'un bouchon verseur en forme d'entonnoir parfaitement adapté à son col. Un geste responsable, indissociable d'un objet créé pour durer.\n\nLE DÉTAIL HERMÈS\nInitialement créé pour être offert aux invités à l'inauguration du premier magasin Hermès de New York le 1er août 1

# Load Prompt

In [7]:
PROMPT_FOLDER_PATH = os.path.join(os.getcwd(), 'prompts')
assert os.path.exists(PROMPT_FOLDER_PATH)

In [8]:
def load_prompts(prompt_folder_path):
    with open(os.path.join(prompt_folder_path, "system_prompt.txt"), "r") as f:
        system_prompt = f.read()
    with open(os.path.join(prompt_folder_path, "human_prompt.txt"), "r") as f:
        human_prompt = f.read()
    return system_prompt, human_prompt

In [9]:
copywriting_system_prompt, copywriting_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "Copywriting"))
brand_review_system_prompt, brand_review_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "BrandReview"))
copywriting_review_system_prompt, copywriting_review_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "CopywritingReview"))
tov_review_system_prompt, tov_review_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "TOVReview"))
editor_system_prompt, editor_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "Editor"))
role =  open(os.path.join(PROMPT_FOLDER_PATH, "Role", "role.txt"), "r").read()


# Writer Chain

In [10]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter


# Model Selection
model = ChatOpenAI(model_name="gpt-4-turbo")

# Output parser creation
class Copywriting(BaseModel):
    generated_text: str = Field(description="The written product detail page.")

class Feedback(BaseModel):
    feedback: str = Field(description="The feedback on the copy.")

class Edition(BaseModel):
    edited_text: str = Field(description="The edited and improved version of the product detail page.")

copywriting_output_parser = JsonOutputParser(pydantic_object=Copywriting)
review_output_parser = JsonOutputParser(pydantic_object=Feedback)
edition_output_parser = JsonOutputParser(pydantic_object=Edition)

# Chains
## Copywriting
copywriting_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(copywriting_system_prompt),
        HumanMessagePromptTemplate.from_template(copywriting_human_prompt)
    ],
    input_variables=["role", "persona", "language", "product_data", "brand_knowledge", "copywriting_guidelines", "platform_specs"],
    partial_variables={"format_instructions": copywriting_output_parser.get_format_instructions()}
)

copywriting_chain = copywriting_prompt | model | copywriting_output_parser

## Brand Review
brand_review_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(brand_review_system_prompt),
        HumanMessagePromptTemplate.from_template(brand_review_human_prompt)
    ],
    input_variables=["role", "brand_knowledge", "generated_text"],
    partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
)
brand_review_chain = brand_review_prompt | model | review_output_parser

## Copywriting Review
copywriting_review_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(copywriting_review_system_prompt),
        HumanMessagePromptTemplate.from_template(copywriting_human_prompt)
    ],
    input_variables=["role", "copywriting_guidelines", "generated_text"],
    partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
)
copywriting_review_chain = copywriting_review_prompt | model | review_output_parser

## TOV Review 
def format_examples(example_dict):
    formated_text = ""
    for product in example_dict.keys():
        formated_text += str(product.upper()) + "\n"
        formated_text +=  str(example_dict[product]) + "\n"
        formated_text += "-" * 10 + "\n"
    return formated_text
        
tov_review_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(tov_review_system_prompt),
        HumanMessagePromptTemplate.from_template(tov_review_human_prompt)
    ],
    input_variables=["role", "reference_examples", "generated_text"],
    partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
)
tov_review_chain = tov_review_prompt | model | review_output_parser

## Edition
edition_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(editor_system_prompt),
        HumanMessagePromptTemplate.from_template(editor_human_prompt)
    ],
    input_variables=["role", "perona", "generated_text", "feedback", "brand_knowledge", "copywriting_guidelines", "reference_examples", "platform_specs"], 
    partial_variables={"format_instructions": edition_output_parser.get_format_instructions()}
)
edition_chain = edition_prompt | model | edition_output_parser

In [11]:
def format_feedback(reviews):
    brand_review, copywriting_review, tov_review = reviews
    return f"""Brand Feedback:
{brand_review}

Copywriting Feedback:
{copywriting_review}

Tone of Voice Feedback:
{tov_review}

"""

write_product_description_chain = (
    RunnablePassthrough.assign(generated_text=copywriting_chain)
    | RunnablePassthrough.assign(reference_examples=itemgetter("reference_examples") | RunnableLambda(format_examples))
    | RunnablePassthrough.assign(
        brand_review=brand_review_chain | itemgetter("feedback"),
        copywriting_review=copywriting_review_chain | itemgetter("feedback"),
        tov_review=tov_review_chain | itemgetter("feedback")
    ) | RunnablePassthrough.assign(feedback=itemgetter("brand_review", "copywriting_review", "tov_review") | RunnableLambda(format_feedback))
    | RunnablePassthrough.assign(edited_text=edition_chain | itemgetter("edited_text"))
    | itemgetter("edited_text")
)

# Write Master

## In French

In [12]:
write_product_description_chain.invoke({
    "role": role, 
    "persona": persona, 
    "language": "fr",
    "product_name": list(product_data["fr"].keys())[0],
    "product_data": product_data["fr"][list(product_data["fr"].keys())[0]],
    "brand_knowledge": brand_knowledge,
    "copywriting_guidelines": copywriting_guidelines,
    "reference_examples": product_data["fr"],
    "platform_specs": hermes_specs,    
})

"LA CRÉATION\nConçue par Christine Nagel, Galop d'Hermès incarne l'esprit artisanal et la maîtrise méticuleuse d'Hermès, évoquant une tradition de savoir-faire inégalée. « En me plongeant dans l'univers d'Hermès, j'ai redécouvert la féminité du cuir et l'ai harmonieusement associée à la rose, tissant un lien entre tradition et modernité », confie Christine Nagel.\n\nLES NOTES OLFACTIVES\nGalop d'Hermès déploie un bouquet floral et cuiré, où la rose voluptueuse et sensuelle s'entrelace avec la douceur du cuir et le caractère épicé du safran, offrant une fragrance audacieuse et raffinée, qui célèbre le patrimoine équestre de la maison.\n\nL'OBJET\nLe flacon-étrier, conçu par Philippe Mouquet, réinterprète le patrimoine équestre d'Hermès, fusionnant fonctionnalité et esthétique. Ce design, inspiré de l'étrier iconique, magnifie l'essence de Galop d'Hermès et enrichit le récit visuel par son élégance intemporelle.\n\nL'ÉTHIQUE\nEngagé en faveur de la durabilité, le flacon est rechargeable 

## In English

In [13]:
write_product_description_chain.invoke({
    "role": role, 
    "persona": persona, 
    "language": "en",
    "product_name": list(product_data["en"].keys())[0],
    "product_data": product_data["en"][list(product_data["en"].keys())[0]],
    "brand_knowledge": brand_knowledge,
    "copywriting_guidelines": copywriting_guidelines,
    "reference_examples": product_data["en"],
    "platform_specs": hermes_specs,    
})

"LA CRÉATION\nEvoking an olfactory journey to Aswan's verdant garden isles, Un Jardin sur le Nil eau de toilette embodies the Nile's tales of life and abundance. Master perfumer Jean-Claude Ellena crafted this fragrance in 2005 as a tribute to the generous spirit and lush fertility of Egypt, infusing it with Hermès' enduring commitment to artisanal excellence and creative freedom.\n\nLES NOTES OLFACTIVES\nA symphony of scents awaits with crisp green mango, intertwined with the subtle elegance of lotus flowers and the deep, earthy tones of sycamore. These notes harmonize to echo the vibrant life along the Nile, inviting a sensory exploration rich in freshness and complexity.\n\nL'OBJET\nThe bottle, a masterpiece designed by Fred Rawyler, reflects the lanterns of horse-drawn carriages with its delicate green gradient, mirroring the lush banks of the Nile. Accompanied by a box adorned with a lotus illustration by Veronique de Mareuil, each detail celebrates the artisanal craftsmanship and

---

# Get Product Details

In [15]:
product_data

{'fr': {'Galop': "LA CRÉATION\nSigné par le parfumeur d'Hermès Christine Nagel en 2016, Galop d'Hermès s'inspire de l'univers équestre et incarne la liberté en mouvement : « Chez Hermès, j'ai découvert toute la féminité du cuir. J'ai composé Galop d'Hermès comme un tableau avec deux matières emblématiques d'Hermès et de la parfumerie : le cuir et la rose. »\n\nLES NOTES OLFACTIVES\nFloral et cuiré, le parfum Galop d'Hermès développe les notes charnues de la rose et celles souples du cuir, au côté d'un safran racé.\n\nL'OBJET\nDessiné par Philippe Mouquet, le flacon-étrier puise avec modernité dans la tradition équestre et l'histoire de la Maison.\n\nL'ÉTHIQUE\nLe flacon se remplit grâce à une recharge de 125 ml munie d'un bouchon verseur en forme d'entonnoir parfaitement adapté à son col. Un geste responsable, indissociable d'un objet créé pour durer.\n\nLE DÉTAIL HERMÈS\nInitialement créé pour être offert aux invités à l'inauguration du premier magasin Hermès de New York le 1er août 1

In [24]:
product_description_system_prompt = """{role}

You are helping the Hermès Product Detail Page copywriting team. They told you they need you to provide them with product description, object description and additional information that captures the information in existing product detail pages.
"""

product_description_human_prompt = """Please provide a product description for {product_name} in {language} based on the existing product detail page:
<product_detail_page>
{product_detail_page}
</product_detail_page>

The product description should capture all the necessary information that would help the Hermès copywriters to write new product detail pages for this product. 
You must structure this info into 3 sections : product description, object description and additional info (for additional info like quotes and key dates). 
Use short and concise sentences and focus on the information.
Make sure all the necessary info to write the product detail page is in your descriptions. 

{format_instructions}
"""

In [25]:
class ProductDetails(BaseModel):
    product_description: str = Field(description="The description of the product")
    object_description: str = Field(description="The description of the object")
    additional_info: str = Field(description="Additional info about the product")
    
product_description_output_parser = JsonOutputParser(pydantic_object=ProductDetails)

product_description_prompt = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template(product_description_system_prompt),
        HumanMessagePromptTemplate.from_template(product_description_human_prompt)
    ],
    input_variables=["role", "persona", "product_name", "language", "product_detail_page"],
    partial_variables={"format_instructions": product_description_output_parser.get_format_instructions()}
)

product_description_chain = product_description_prompt | model | product_description_output_parser

In [26]:
def describe_product(product_name, language):
    return product_description_chain.invoke({
        "role": role,
        "product_name": product_name,
        "language": language,
        "product_detail_page": product_data[language][product_name]
    })

In [27]:
product_description = {"en": {}, "fr": {}}

for lang in product_description:
    for product in product_data[lang]:
        product_desc = describe_product(product, lang)
        product_description[lang][product] = product_desc

In [28]:
product_description

{'en': {'Galop': {'product_description': 'Created in 2005, Un Jardin sur le Nil eau de toilette evokes an olfactory journey to the garden islands of Aswan, Egypt. This fragrance, crafted by perfumer Jean-Claude Ellena, encapsulates the essence of the Nile, symbolizing life and generosity. It features a blend of green mango, lotus, and sycamore, presenting a green and woody profile.',
   'object_description': "The bottle, designed by Fred Rawyler, draws inspiration from horse-drawn carriage lanterns and features a delicate green gradient glass that echoes the Nile's banks. The packaging, adorned with a lotus flower illustration by Veronique de Mareuil, complements the theme.",
   'additional_info': "Part of the Parfums-Jardins collection, Un Jardin sur le Nil is designed to embody the soul of a location, inspired by the perfumer's vision and Hermès' annual creative theme. This collection is intended for those seeking olfactory experiences that offer freshness, dreams, or an escape."},
 

In [33]:
for lang in product_description:
    for product in product_description[lang]:
        with open(os.path.join(DATA_FOLDER_PATH, "product_details", lang, "product_description", product.lower().replace(' ', '_').replace('è', 'e') + '.txt'), "w") as f:
            f.write(product_description[lang][product]["product_description"])
        with open(os.path.join(DATA_FOLDER_PATH, "product_details", lang, "object_description", product.lower().replace(' ', '_').replace('è', 'e') + '.txt'), "w") as f:
            f.write(product_description[lang][product]["object_description"])
        with open(os.path.join(DATA_FOLDER_PATH, "product_details", lang, "additional_info", product.lower().replace(' ', '_').replace('è', 'e') + '.txt'), "w") as f:
            f.write(product_description[lang][product]["additional_info"])

---

# Retailer Perso

In [38]:
retailer_product_data = {
    retailer: {
        lang: {
            product.capitalize().replace('_', ' ')[:-4]: open(os.path.join(DATA_FOLDER_PATH, "retailer_product_data", retailer, lang, product), "r").read() for product in os.listdir(os.path.join(DATA_FOLDER_PATH, "retailer_product_data", retailer, lang))
        } for lang in ["en", "fr"]
    } for retailer in os.listdir(os.path.join(DATA_FOLDER_PATH, "retailer_product_data"))
}

In [40]:
def format_customization_example(product):
    product = product.lower().replace('_', ' ')
    if not ".txt" in product:
        product += ".txt"
    return f"SOURCE TEXT:\n{retailer_product_data['hermes']}\n\nTARGET TEXT:\n{retailer_product_data['sephora']}"

In [37]:
retailer_product_data

{'sephora': {'en': {'Galop': "LA CRÉATION\nSigné par le parfumeur d'Hermès Christine Nagel en 2016, Galop d'Hermès s'inspire de l'univers équestre et incarne la liberté en mouvement : « Chez Hermès, j'ai découvert toute la féminité du cuir. J'ai composé Galop d'Hermès comme un tableau avec deux matières emblématiques d'Hermès et de la parfumerie : le cuir et la rose. »\n\nLES NOTES OLFACTIVES\nFloral et cuiré, le parfum Galop d'Hermès développe les notes charnues de la rose et celles souples du cuir, au côté d'un safran racé.\n\nL'OBJET\nDessiné par Philippe Mouquet, le flacon-étrier puise avec modernité dans la tradition équestre et l'histoire de la Maison.\n\nL'ÉTHIQUE\nLe flacon se remplit grâce à une recharge de 125 ml munie d'un bouchon verseur en forme d'entonnoir parfaitement adapté à son col. Un geste responsable, indissociable d'un objet créé pour durer.\n\nLE DÉTAIL HERMÈS\nInitialement créé pour être offert aux invités à l'inauguration du premier magasin Hermès de New York l

## Retailer Customization 

In [68]:
from functions.utils import load_prompts

customization_system_prompt, customization_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "Customization"))
edition_customization_system_prompt, edition_customization_human_prompt = load_prompts(os.path.join(PROMPT_FOLDER_PATH, "CustomizationEditor"))

In [78]:
# Output parser creation
class Customization(BaseModel):
    customized_text: str = Field(description="The retailer customized product detail page.")

class Feedback(BaseModel):
    feedback: str = Field(description="The feedback on the copy.")

class Edition(BaseModel):
    edited_text: str = Field(description="The edited and improved version of the customized product detail page.")

customization_output_parser = JsonOutputParser(pydantic_object=Customization)
review_output_parser = JsonOutputParser(pydantic_object=Feedback)
edition_output_parser = JsonOutputParser(pydantic_object=Edition)

# Chains
## Copywriting
customization_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(customization_system_prompt),
        HumanMessagePromptTemplate.from_template(customization_human_prompt)
    ],
    input_variables=["role", "persona", "language", "existing_product_dp", "brand_knowledge", "copywriting_guidelines", "reference_examples", "platform_specs"],
    partial_variables={"format_instructions": customization_output_parser.get_format_instructions()}
)

customization_chain = customization_prompt | model | customization_output_parser

## Brand Review
brand_review_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(brand_review_system_prompt),
        HumanMessagePromptTemplate.from_template(brand_review_human_prompt)
    ],
    input_variables=["role", "brand_knowledge", "generated_text"],
    partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
)
brand_review_chain = brand_review_prompt | model | review_output_parser

## Copywriting Review
copywriting_review_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(copywriting_review_system_prompt),
        HumanMessagePromptTemplate.from_template(copywriting_human_prompt)
    ],
    input_variables=["role", "copywriting_guidelines", "generated_text"],
    partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
)
copywriting_review_chain = copywriting_review_prompt | model | review_output_parser

    ## TOV & Platform Specs Review 
def format_examples(example_dict):
    formated_text = ""
    for language in ["en", "fr"]:
        formated_text += f"\nLanguage: {language}\n\n"
        for product in example_dict["hermes"][language]:
            formated_text += f"Original text:\n{example_dict['hermes'][language][product]}\n\nAdapted text:\n{example_dict['sephora'][language][product]}\n\n"
    return formated_text
        
tov_review_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(tov_review_system_prompt),
        HumanMessagePromptTemplate.from_template(tov_review_human_prompt)
    ],
    input_variables=["role", "reference_examples", "generated_text"],
    partial_variables={"format_instructions": review_output_parser.get_format_instructions()}
)
tov_review_chain = tov_review_prompt | model | review_output_parser

## Edition
edition_customization_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(edition_customization_system_prompt),
        HumanMessagePromptTemplate.from_template(edition_customization_human_prompt)
    ],
    input_variables=["role", "persona", "generated_text", "feedback", "brand_knowledge", "copywriting_guidelines", "reference_examples", "existing_product_dp", "platform_specs"], 
    partial_variables={"format_instructions": edition_output_parser.get_format_instructions()}
)
edition_chain = edition_customization_prompt | model | edition_output_parser

def format_feedback(reviews):
    brand_review, copywriting_review, tov_review = reviews
    return f"""Brand Feedback:
{brand_review}

Copywriting Feedback:
{copywriting_review}

Tone of Voice and Platform specs compliance Feedback:
{tov_review}

"""

customize_product_description_chain = (
    RunnablePassthrough.assign(generated_text=customization_chain)
    | RunnablePassthrough.assign(reference_examples=itemgetter("reference_examples") | RunnableLambda(format_examples))
    | RunnablePassthrough.assign(
        brand_review=brand_review_chain | itemgetter("feedback"),
        copywriting_review=copywriting_review_chain | itemgetter("feedback"),
        tov_review=tov_review_chain | itemgetter("feedback")
    ) | RunnablePassthrough.assign(feedback=itemgetter("brand_review", "copywriting_review", "tov_review") | RunnableLambda(format_feedback))
    | RunnablePassthrough.assign(edited_text=customization_chain)
    | itemgetter("edited_text")
)   



In [66]:
if product_name in product_data[language]:
    product_details = product_data[language][product_name]
else:
    def format_product_details(product_detail_list):
        return  f"PRODUCT DESCRIPTION:\n{product_detail_list[0]}\n\nOBJECT DESCRIPTION:\n{product_detail_list[1]}\n\nADDITIONAL INFO:\n{product_detail_list[2]}"
    product_details = format_product_details(product_details)

        
customize_product_description_chain.invoke({
    "role": role, 
    "persona": persona, 
    "language": language,
    "product_name": product_name,
    "product_data": product_details,
    "brand_knowledge": brand_knowledge,
    "copywriting_guidelines": copywriting_guidelines,
    "reference_examples": product_data[language],
    "platform_specs": sephora_specs,   
    "existing_product_dp": product_data[language][product_name] 
})

NameError: name 'product_name' is not defined

In [79]:
language = "fr"
product = "terre"

In [80]:
customize_product_description_chain.invoke({
    "role": role, 
    "persona": persona, 
    "language": language,
    "product_name": product,
    "brand_knowledge": brand_knowledge,
    "copywriting_guidelines": copywriting_guidelines,
    "reference_examples": retailer_product_data,
    "platform_specs": sephora_specs,   
    "existing_product_dp": product_data[language]["Terre d Hermès"], 
    "product_data": retailer_product_data["sephora"][language]["Terre d hermes"] 
})

{'customized_text': "« La terre est une harmonie silencieuse pour ceux qui écoutent. » – Jean-Claude Ellena.\n\nTerre d'Hermès évoque un voyage profond à travers les éléments, un dialogue entre l'homme et la nature, révélant des liens presque oubliés avec la terre qui nous nourrit et nous soutient. Cette fragrance, créée par Jean-Claude Ellena en 2006 et revisité par Christine Nagel depuis 2016, explore les nouveaux possibles, en alliant la fraîcheur du pamplemousse à la robustesse du cèdre et à un silex vibrant d'énergie.\n\nLe flacon lui-même est un manifeste de cette connexion entre ciel et terre. Sa base, ornée d'un H orange, s'ancre dans la terre, tandis que ses épaules métalliques reflètent la lumière du ciel. Ce design, œuvre de Philippe Mouquet, s'inspire des nécessaires de voyage conservés au musée Émile Hermès, rappelant ainsi l'importance du voyage dans l'ADN de la maison.\n\nTerre d'Hermès est plus qu'un parfum, c'est une fenêtre ouverte sur notre monde, une célébration des

In [62]:
product_data

{'fr': {'Galop': "LA CRÉATION\nSigné par le parfumeur d'Hermès Christine Nagel en 2016, Galop d'Hermès s'inspire de l'univers équestre et incarne la liberté en mouvement : « Chez Hermès, j'ai découvert toute la féminité du cuir. J'ai composé Galop d'Hermès comme un tableau avec deux matières emblématiques d'Hermès et de la parfumerie : le cuir et la rose. »\n\nLES NOTES OLFACTIVES\nFloral et cuiré, le parfum Galop d'Hermès développe les notes charnues de la rose et celles souples du cuir, au côté d'un safran racé.\n\nL'OBJET\nDessiné par Philippe Mouquet, le flacon-étrier puise avec modernité dans la tradition équestre et l'histoire de la Maison.\n\nL'ÉTHIQUE\nLe flacon se remplit grâce à une recharge de 125 ml munie d'un bouchon verseur en forme d'entonnoir parfaitement adapté à son col. Un geste responsable, indissociable d'un objet créé pour durer.\n\nLE DÉTAIL HERMÈS\nInitialement créé pour être offert aux invités à l'inauguration du premier magasin Hermès de New York le 1er août 1